SCRAPER για TEASERS PAGES (SEARCH) στη Ναυτεμπορική

Βιβλιοθήκη

In [1]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site και να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

Αίτημα στον server

In [2]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

Function για εξόρυξη των elements μιας σελίδας με teasers από τη soup

In [3]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = []
  for article in articlesL:
    try:
      title = article.find("h3", {"class": "item-title"}).text.strip()
    except:
      title=""
    try:
      article_url = article.find("h3", {"class": "item-title"}).find('a')["href"]
    except:
      article_url=""
    try:
      time = article.find("time", {"class":"item-published"})
      datetime_str = time.text
      date_str = datetime_str.split("-")[0]
      time_str = datetime_str.split("-")[1]
    except:
      date_str = ""
      time_str = ""
    try:
      section = article.find("span", {"class":"category fw-bold text-primary"}).text.strip()
    except:
      section = ""
    # Δημιουργία λεξικού με τα data του άρθρου
    articleD = {"title": title, "article_url": article_url, "date": date_str, "time": time_str, "section": section}

    # Προσθήκη των data του κάθε άρθρου στη λίστα των data των άρθρων
    articles_dataL.append(articleD)

  return articles_dataL

Function για το σκραπάρισμα σελίδων από την αναζήτηση του naftemporiki.gr

In [9]:
def scrape_naftemporiki_teaser_pages(search_term, start_page_nbr, end_page_nbr):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει
    # η δομή των teasers pages urls του naftemporiki.gr
    # if "search" in scrape_url:
    #   search_term = scrape_url.split("/")[4]
    #   if len(scrape_url.split("/")) > 7:
    #     search_params = scrape_url.split("/")[-1]
    #   else:
    #     search_params = ""
    #   url = f'https://www.kathimerini.gr/search/{search_term}/page/{page_nbr}/{search_params}'
    # else:
    #   category = scrape_url.split("/")[3]
    #   url = f"https://www.kathimerini.gr/{category}/page/{page_nbr}/"
    url = f"https://www.naftemporiki.gr/page//{page_nbr}/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+"

    print(f"\rScraping page {page_nbr}: {url}" + " "*2, end="")

    # φέρνω τη σούπα της σελίδας με το function που έφτιαξα παραπάνω στο βήμα 2
    soup = get_webpage_soup(url)

    # βάζω τον κώδικα με τον οποίο βρίσκω όλα τα blocks των άρθρων - το έφτιαξα στο βήμα 6
    # articlesL = soup.find_all("div", {"class":"nx-article"})
    articlesL = soup.find("div", {"class":"section-body mb-3"}).find_all("div",{"class":"g-col-12"})

    # εξορύσσω τα δεδομένα όλων των άρθρων της σελίδας με το function που έφτιαξα στο βήμα 8
    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    # προσθέτω τα δεδομένα των άρθρων της σελίδας στη λίστα με τα δεδομένα όλων των άρθρων - τη δημιούργησα στην αρχή αυτού του function
    all_articles_dataL.extend(page_articles_dataL)

    # Τυχαία καθυστέρηση μεταξύ 1 και 3 δευτερολέπτων ανάμεσα στα page scrapes
    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  return df

### ΔΟΚΙΜΗ ΛΕΙΤΟΥΡΓΙΑΣ

In [10]:
df = scrape_naftemporiki_teaser_pages("Τέμπη", 4, 14) # όρος αναζήτησης, o αριθμός της πρώτης σελίδας του pagination που θέλουμε, ο αριθμός της τελευταίας σελίδας του pagination που θέλουμε
df

Scraping page 4: https://www.naftemporiki.gr/page//4/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+  https://www.naftemporiki.gr/page//4/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+ was scraped successfully
Scraping page 5: https://www.naftemporiki.gr/page//5/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+  https://www.naftemporiki.gr/page//5/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+ was scraped successfully
Scraping page 6: https://www.naftemporiki.gr/page//6/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+  https://www.naftemporiki.gr/page//6/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+ was scraped successfully
Scraping page 7: https://www.naftemporiki.gr/page//7/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+  https://www.naftemporiki.gr/page//7/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+ was scraped successfully
Scraping page 8: https://www.naftemporiki.gr/page//8/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+  https://www.naftemporiki.gr/page//8/?s=%CF%84%CE%B5%CE%BC%CF%80%CE%B7+ was scraped successfully
Scraping page 9: https://www.naftemporiki.gr/page//9/?s=%CF%84%CE%B5%CE%BC%

title  \
0    Ντ. Μπακογιάννη: Ως βουλευτής, ζητώ συγγνώμη α...   
1    Βουλή: Αντιπαράθεση Χρυσοχοΐδη με ΚΚΕ και Νέα ...   
2    Αλ. Χαρίτσης για Τέμπη: Ο πρωθυπουργός «έκανε ...   
3    Βουλή: Νέος γύρος σφοδρής αντιπαράθεσης – Κόντ...   
4    Τέμπη: Ξανά στους δρόμους μαθητές και φοιτητές...   
..                                                 ...   
325  Το χαρτοφυλάκιο Μεταφορών, η Ναυτιλία, τα Τέμπ...   
326  Κ. Αρβανίτης: Με το χαρτοφυλάκιο Τζιτζικώστα ε...   
327  Ν. Παππάς για Τέμπη: Το μπάζωμα έγινε με χρήμα...   
328  ΣΥΡΙΖΑ-Τέμπη: Αίτημα στην Εισαγγελία για νέες ...   
329  Τέμπη: Αποφυλακίστηκε ο σταθμάρχης Λάρισας – Σ...   

                                           article_url         date    time  \
0    https://www.naftemporiki.gr/politics/1930155/n...  06/03/2025    21:19   
1    https://www.naftemporiki.gr/politics/1930100/v...  06/03/2025    18:55   
2    https://www.naftemporiki.gr/politics/1929993/a...  06/03/2025    16:10   
3    https://www.naftemporiki.gr/politics/1929961/v...  06/03/2025    15:39   
4    https://www.naftemporiki.gr/society/1929751/te...  06/03/2025    07:59   
..                                                 ...          ...     ...   
325  https://www.naftemporiki.gr/opinion/1770541/i-...  19/09/2024    09:29   
326  https://www.naftemporiki.gr/politics/1768615/k...  17/09/2024    12:54   
327  https://www.naftemporiki.gr/politics/1766858/n...  15/09/2024    13:09   
328  https://www.naftemporiki.gr/politics/1754140/s...  31/08/2024    14:47   
329  https://www.naftemporiki.gr/society/1730502/te...  01/08/2024    08:54   

      section  
0    Πολιτική  
1    Πολιτική  
2    Πολιτική  
3    Πολιτική  
4    Κοινωνία  
..        ...  
325   Απόψεις  
326  Πολιτική  
327  Πολιτική  
328  Πολιτική  
329  Κοινωνία  

[330 rows x 5 columns]

ΑΠΟΘΗΚΕΥΣΗ ΤΟΥ CSV

In [12]:
df.to_csv('/content/drive/MyDrive/naft_tempi_teaser_articles.csv', index=False)

In [13]:
df.to_csv('naft_tempi_teaser_articles.csv', index=False)

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
from google.colab import files
files.download('naft_tempi_teaser_articles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#scraping τα άρθρα

In [19]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site ώστε να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

from tqdm import tqdm # για να βάλω progress bar με το οποίο θα παρακολουθώ την πορεία του scraping

In [20]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    #print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [21]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου απόο όλη τη σούπα της σελίδας
  article_block = soup.find("article")
  # article url
  article_url = article_url # το παίρνω από το arguement του function
  # title
  try:
    title = soup.find("h1" , class_=lambda c: c and "title" in c).text.strip()
  except:
    title=""
  # datetime
  try:
    datetime_str = soup.find("time")["datetime"]
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("+")[0]
  except:
    date_str = ""
    time_str = ""
  # section
  try:
    section = soup.find("div", class_="post-category blue").text.strip().replace("\n", "")
  except:
    section = ""
  # author
  try:
    author = soup.find("div", class_="post-author").text.strip()
  except:
    author = ""
  # excerpt
  try:
   excerpt = soup.find("div", class_="post-content mb-25").find("h3").text.strip()
   excerpt = excerpt.replace("\xa0", "").strip()
  except:
    excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", class_= "post-content mb-25").find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_link = soup.find("img", src=True)["src"]
  except:
    feat_img_link=""
  # feature image caption
  try:
    image_caption = soup.find("div", class_="entry-summary").find("p").text.strip()
  except:
    image_caption=""

  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "text_body": text_body, "feat_img_link": feat_img_link, "feat_img_cation": image_caption}

  return articleD

In [22]:
def scrape_naftemporiki_articles_pages(naftemporiki_teasers_df):
  # δημιουργία κενής λίστας στην οποία θα προσθέτουμε ένα-ένα τα λεξικά με τα δεδομένα των άρθρων που θα σκραπάρουμε
  all_articles_dataL = []
  # λούπα σε κάθε row του teasers_df για να παίρνουμε ένα-ένα τα urls των άρθρων και να το σκραπάρουμε
  # χρησιμοποιούμε το tqdm για να βλέπουμε σε ένα progress bar πού βρισκόμαστε και πόσος χρόνος απομένει ακόμα
  for idx, row in tqdm(naftemporiki_teasers_df.iterrows(), total=naftemporiki_teasers_df.shape[0], desc="Processing rows"):
    # παίρνουμε το url του άρθρου που θα σκραπάρουμε από τη στήλη "article_url" του row του teasers_df στο οποίο βρίσκεται η for loop
    article_url = row['article_url']
    # scrape της σελίδας του url του κάθε άρθρου με το function που φτιάξαμε στο βήμα 2
    soup = get_webpage_soup(article_url)
    # δημιουργία του λεξικού με τα δεδομένα του κάθε άρθρου με το function που φτιάξαμε στο βήμα 9
    articleD = get_article_elements_from_soup(soup, article_url)
    # προσθήκη του λεξικού με τα δεδομένα του κάθε άρθρου στη λίστα που δημιουργήσαμε στη αρχή
    all_articles_dataL.append(articleD)
    # καθυστέρηση ανάμεσα στα requests στο server
    delay = random.uniform(1, 3)
    time.sleep(delay)
  # δημιουργία dataframe από τη λίστα των λεξικών με τα δεδομένα κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [23]:
naftemporiki_teasers_df = pd.read_csv("/content/naft_tempi_teaser_articles.csv")
naftemporiki_teasers_df.head()

title  \
0  Ντ. Μπακογιάννη: Ως βουλευτής, ζητώ συγγνώμη α...   
1  Βουλή: Αντιπαράθεση Χρυσοχοΐδη με ΚΚΕ και Νέα ...   
2  Αλ. Χαρίτσης για Τέμπη: Ο πρωθυπουργός «έκανε ...   
3  Βουλή: Νέος γύρος σφοδρής αντιπαράθεσης – Κόντ...   
4  Τέμπη: Ξανά στους δρόμους μαθητές και φοιτητές...   

                                         article_url         date    time  \
0  https://www.naftemporiki.gr/politics/1930155/n...  06/03/2025    21:19   
1  https://www.naftemporiki.gr/politics/1930100/v...  06/03/2025    18:55   
2  https://www.naftemporiki.gr/politics/1929993/a...  06/03/2025    16:10   
3  https://www.naftemporiki.gr/politics/1929961/v...  06/03/2025    15:39   
4  https://www.naftemporiki.gr/society/1929751/te...  06/03/2025    07:59   

    section  
0  Πολιτική  
1  Πολιτική  
2  Πολιτική  
3  Πολιτική  
4  Κοινωνία

In [25]:
naft_articles_df = scrape_naftemporiki_articles_pages(naftemporiki_teasers_df)
naft_articles_df

Processing rows: 100%|██████████| 330/330 [15:29<00:00,  2.82s/it]


Scraping completed. Fetshed 330 articles.


article_url  \
0    https://www.naftemporiki.gr/politics/1930155/n...   
1    https://www.naftemporiki.gr/politics/1930100/v...   
2    https://www.naftemporiki.gr/politics/1929993/a...   
3    https://www.naftemporiki.gr/politics/1929961/v...   
4    https://www.naftemporiki.gr/society/1929751/te...   
..                                                 ...   
325  https://www.naftemporiki.gr/opinion/1770541/i-...   
326  https://www.naftemporiki.gr/politics/1768615/k...   
327  https://www.naftemporiki.gr/politics/1766858/n...   
328  https://www.naftemporiki.gr/politics/1754140/s...   
329  https://www.naftemporiki.gr/society/1730502/te...   

                                                 title        date      time  \
0    Ντ. Μπακογιάννη: Ως βουλευτής, ζητώ συγγνώμη α...  2025-03-06  21:19:03   
1    Βουλή: Αντιπαράθεση Χρυσοχοΐδη με ΚΚΕ και Νέα ...  2025-03-06  18:55:18   
2    Αλ. Χαρίτσης για Τέμπη: Ο πρωθυπουργός «έκανε ...  2025-03-06  16:10:10   
3    Βουλή: Νέος γύρος σφοδρής αντιπαράθεσης – Κόντ...  2025-03-06  15:39:24   
4    Τέμπη: Ξανά στους δρόμους μαθητές και φοιτητές...  2025-03-06  07:59:18   
..                                                 ...         ...       ...   
325  Το χαρτοφυλάκιο Μεταφορών, η Ναυτιλία, τα Τέμπ...  2024-09-19  09:29:20   
326  Κ. Αρβανίτης: Με το χαρτοφυλάκιο Τζιτζικώστα ε...  2024-09-17  12:54:06   
327  Ν. Παππάς για Τέμπη: Το μπάζωμα έγινε με χρήμα...  2024-09-15  13:09:24   
328  ΣΥΡΙΖΑ-Τέμπη: Αίτημα στην Εισαγγελία για νέες ...  2024-08-31  14:47:05   
329  Τέμπη: Αποφυλακίστηκε ο σταθμάρχης Λάρισας – Σ...  2024-08-01  08:54:18   

      section                                             author  \
0    Πολιτική                                                      
1    Πολιτική                                                      
2    Πολιτική                                                      
3    Πολιτική                                                      
4    Κοινωνία                                                      
..        ...                                                ...   
325   Απόψεις  Νατάσα Στασινού                               ...   
326  Πολιτική                                                      
327  Πολιτική                                                      
328  Πολιτική                                                      
329  Κοινωνία                                                      

                                               excerpt  \
0    «Ειλικρινή συγγνώμη» στους γονείς των παιδιών ...   
1    «Η υπόθεση των Τεμπών με τα 57 θύματα είναι μι...   
2    Για έναν πρωθυπουργό «θρασύ» και «αλαζόνα» έκα...   
3    Πυρά και αντεγκλήσεις μεταξύ του υπουργού Επικ...   
4    Νέο συλλαλητήριο για τα Τέμπη διοργανώνουν την...   
..                                                 ...   
325                                                      
326  Δηκτικά σχολίασε ο επικεφαλής της ευρωομάδας τ...   
327                                                      
328  Αίτημα «για διερεύνηση νέων ποινικών αδικημάτω...   
329                                                      

                                             text_body  \
0    «Όλοι μας και οι 300 βουλευτές, όπως και ολόκλ...   
1    Η ασφάλεια, είπε ο υπουργός «είναι κοινωνικό α...   
2    Συγκεκριμένα, καταλόγισε στον πρωθυπουργό ότι ...   
3    Μιλώντας στη Βουλή κατά τη συζήτηση επί της πρ...   
4    «Μέχρι η δικαίωση να ‘ρθει εδώ θα μείνουμε. Όλ...   
..                                                 ...   
325  Με την πείρα πέντε δεκαετιών δραστηριοποίησης ...   
326  «Με τη βοήθεια της Ούρσουλα Φον ντερ Λάιεν και...   
327  Για έγκλημα και προσπάθεια συγκάλυψής του έκαν...   
328  Συγκεκριμένα, το αίτημα θέα καταθέσουν στην Ει...   
329  Σε κατ’ οίκον περιορισμό βρίσκεται πλέον ο στα...   

                                         feat_img_link  \
0    https://www.naftemporiki.gr/wp-content/themes/...   
1    https://www.naftemporiki.gr/wp-content/themes/...   


In [26]:
naft_articles_df

article_url  \
0    https://www.naftemporiki.gr/politics/1930155/n...   
1    https://www.naftemporiki.gr/politics/1930100/v...   
2    https://www.naftemporiki.gr/politics/1929993/a...   
3    https://www.naftemporiki.gr/politics/1929961/v...   
4    https://www.naftemporiki.gr/society/1929751/te...   
..                                                 ...   
325  https://www.naftemporiki.gr/opinion/1770541/i-...   
326  https://www.naftemporiki.gr/politics/1768615/k...   
327  https://www.naftemporiki.gr/politics/1766858/n...   
328  https://www.naftemporiki.gr/politics/1754140/s...   
329  https://www.naftemporiki.gr/society/1730502/te...   

                                                 title        date      time  \
0    Ντ. Μπακογιάννη: Ως βουλευτής, ζητώ συγγνώμη α...  2025-03-06  21:19:03   
1    Βουλή: Αντιπαράθεση Χρυσοχοΐδη με ΚΚΕ και Νέα ...  2025-03-06  18:55:18   
2    Αλ. Χαρίτσης για Τέμπη: Ο πρωθυπουργός «έκανε ...  2025-03-06  16:10:10   
3    Βουλή: Νέος γύρος σφοδρής αντιπαράθεσης – Κόντ...  2025-03-06  15:39:24   
4    Τέμπη: Ξανά στους δρόμους μαθητές και φοιτητές...  2025-03-06  07:59:18   
..                                                 ...         ...       ...   
325  Το χαρτοφυλάκιο Μεταφορών, η Ναυτιλία, τα Τέμπ...  2024-09-19  09:29:20   
326  Κ. Αρβανίτης: Με το χαρτοφυλάκιο Τζιτζικώστα ε...  2024-09-17  12:54:06   
327  Ν. Παππάς για Τέμπη: Το μπάζωμα έγινε με χρήμα...  2024-09-15  13:09:24   
328  ΣΥΡΙΖΑ-Τέμπη: Αίτημα στην Εισαγγελία για νέες ...  2024-08-31  14:47:05   
329  Τέμπη: Αποφυλακίστηκε ο σταθμάρχης Λάρισας – Σ...  2024-08-01  08:54:18   

      section                                             author  \
0    Πολιτική                                                      
1    Πολιτική                                                      
2    Πολιτική                                                      
3    Πολιτική                                                      
4    Κοινωνία                                                      
..        ...                                                ...   
325   Απόψεις  Νατάσα Στασινού                               ...   
326  Πολιτική                                                      
327  Πολιτική                                                      
328  Πολιτική                                                      
329  Κοινωνία                                                      

                                               excerpt  \
0    «Ειλικρινή συγγνώμη» στους γονείς των παιδιών ...   
1    «Η υπόθεση των Τεμπών με τα 57 θύματα είναι μι...   
2    Για έναν πρωθυπουργό «θρασύ» και «αλαζόνα» έκα...   
3    Πυρά και αντεγκλήσεις μεταξύ του υπουργού Επικ...   
4    Νέο συλλαλητήριο για τα Τέμπη διοργανώνουν την...   
..                                                 ...   
325                                                      
326  Δηκτικά σχολίασε ο επικεφαλής της ευρωομάδας τ...   
327                                                      
328  Αίτημα «για διερεύνηση νέων ποινικών αδικημάτω...   
329                                                      

                                             text_body  \
0    «Όλοι μας και οι 300 βουλευτές, όπως και ολόκλ...   
1    Η ασφάλεια, είπε ο υπουργός «είναι κοινωνικό α...   
2    Συγκεκριμένα, καταλόγισε στον πρωθυπουργό ότι ...   
3    Μιλώντας στη Βουλή κατά τη συζήτηση επί της πρ...   
4    «Μέχρι η δικαίωση να ‘ρθει εδώ θα μείνουμε. Όλ...   
..                                                 ...   
325  Με την πείρα πέντε δεκαετιών δραστηριοποίησης ...   
326  «Με τη βοήθεια της Ούρσουλα Φον ντερ Λάιεν και...   
327  Για έγκλημα και προσπάθεια συγκάλυψής του έκαν...   
328  Συγκεκριμένα, το αίτημα θέα καταθέσουν στην Ει...   
329  Σε κατ’ οίκον περιορισμό βρίσκεται πλέον ο στα...   

                                         feat_img_link  \
0    https://www.naftemporiki.gr/wp-content/themes/...   
1    https://www.naftemporiki.gr/wp-content/themes/...   


In [28]:
naft_articles_df.to_csv('/content/drive/MyDrive//naft_tempi_articles.csv', index=False)

In [29]:
naft_articles_df.to_csv('naft_tempi_articles.csv', index=False)

In [31]:
from google.colab import files
files.download('naft_tempi_articles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>